 Aggregated Residual Transformations for Deep Neural Networks

 Xie, S., Girshick, R., Dollár, P., Tu, Z. and He, K., 2017. Aggregated residual transformations for deep neural networks. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1492-1500)

In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnext101_32x8d', pretrained=True)
model.eval()

Using cache found in C:\Users\Adm/.cache\torch\hub\pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [3]:
from PIL import Image
from torchvision import transforms

In [4]:
filepath = r'E:/Datasets/cor/test/normal/153/CODE153_2023-05-30_21-38-49.jpg'
# filepath = r'/mnt/e/Datasets/cor/test/normal/153/CODE153_2023-05-30_21-38-49.jpg'
input_image = Image.open(filepath)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

In [5]:
# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    print('cuda is available')
    input_batch = input_batch.to('cuda')
    model.to('cuda')

cuda is available


In [6]:
with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-1.2984e+00, -6.6855e-01, -1.8604e+00, -2.8240e+00, -1.2040e+00,
         1.3949e-01, -3.3319e+00, -1.7991e+00, -1.8133e+00, -1.4335e+00,
        -1.2880e+00, -5.3990e-01, -3.1225e+00, -2.9961e+00, -3.7404e+00,
        -3.3879e+00, -2.4940e+00, -5.3142e+00, -1.7284e+00, -2.6742e+00,
        -3.2580e+00, -2.9461e+00, -1.3539e+00, -2.5036e-01, -2.8359e+00,
        -2.7875e+00, -1.4053e-01,  3.5596e-01, -6.9057e-02, -1.1228e+00,
        -1.3965e+00,  1.3184e+00, -3.7975e-01, -3.7084e-01,  4.5963e-01,
        -2.6578e+00, -1.1131e+00, -4.1030e+00,  7.0532e-01, -1.0448e+00,
        -9.4203e-01, -9.3312e-01, -3.5354e+00, -1.3928e+00, -9.2698e-01,
        -2.0861e+00, -2.1894e+00,  8.7755e-01, -2.1977e+00, -1.6447e+00,
        -2.3343e+00,  1.0024e+00,  1.3931e-01,  2.8250e-01, -3.2743e+00,
        -1.6212e+00, -2.4345e+00, -1.8232e+00,  2.0898e-01, -3.2414e-01,
         3.5253e+00, -1.6873e+00, -1.2566e+00, -8.2623e-01, -8.8792e-01,
        -2.5725e-01, -3.3751e-02, -4.4656e-01, -9.3

In [7]:
# Read the categories
with open("docs/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)

for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())


limousine 0.2481282651424408
trailer truck 0.09169013053178787
streetcar 0.0692308098077774
pole 0.06870850175619125
cab 0.06814604997634888
